In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class reviews:
     def open_datasets(self):
        global reviews
        #import reviews dataset csv
        reviews = pd.read_csv("goodreads_train.csv", usecols=['book_id', 'review_text', 'review_id', 'n_votes'])

        #import titles json dataset
        cols = ['book_id', 'title', 'image_url', 'url', 'average_rating', 'description']
        data = []
        file_name = 'goodreads_books.json';
        with open(file_name, encoding='latin-1') as f:
            for line in f:
                doc = json.loads(line)
                lst = [doc['book_id'], doc['title'], doc["image_url"], doc["url"], doc["average_rating"], doc["description"]]
                data.append(lst)

        titles = pd.DataFrame(data=data, columns=cols)

        #import genres json dataset
        cols = ['book_id', 'genres']
        data = []
        file_name = 'goodreads_book_genres_initial.json';

        with open(file_name, encoding='latin-1') as f:
            for line in f:
                doc = json.loads(line)
                lst = [doc['book_id'], doc['genres']]
                data.append(lst)

        genres = pd.DataFrame(data=data, columns=cols)

        #merge and minimize
        titles['book_id']=titles['book_id'].astype(int)
        reviews['book_id']=reviews['book_id'].astype(int)
        genres['book_id']=genres['book_id'].astype(int)
        reviews = pd.merge(reviews, titles, on="book_id")
        reviews = pd.merge(reviews, genres, on="book_id")
        reviews = reviews[reviews['review_text'].str.split().str.len().gt(25)]
        reviews = reviews[reviews['n_votes'].gt(20)]
        
        return reviews

class transform:
    
    def __init__(self, reviews):
        self.reviews = reviews
        
    def transform(self, reviews):
        global tfidf_tokens
        global tfidf_wm
        tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
        # convert th documents into a matrix
        tfidf_wm = tfidfvectorizer.fit_transform(reviews["review_text"]);
        tfidf_wm = tfidf_wm.astype("float32")
        #retrieve the terms found in the corpora
        # if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , it will give same output of get_feature_names() methods)
        tfidf_tokens = tfidfvectorizer.get_feature_names()
        df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)


        df_tfidfvect = df_tfidfvect.astype("float32")
        similarity_matrix = cosine_similarity(df_tfidfvect, df_tfidfvect);

        
        return tfidf_wm, tfidf_tokens

In [44]:
class model:
    
    def __init__(self, reviews, tfidf_wm, tfidf_tokens):
        self.reviews = reviews
        self.tfidf_wm = tfidf_wm
        self.tfidf_tokens = tfidf_tokens
        

    def retrive(self, query, tags):
        # Instead of using fit_transform, you need to first fit 
        # the new document to the TFIDF matrix corpus like this:
        queryTFIDF = TfidfVectorizer().fit(self.tfidf_tokens)
        # Now we can 'transform' this vector into that matrix shape by using the transform function:
        queryTFIDF = queryTFIDF.transform([query])
        # As we transformed our query in a tfidf object
        # we can calculate the cosine similarity in comparison with 
        # our pevious corpora
        cosine_similarities = cosine_similarity(queryTFIDF, self.tfidf_wm).flatten()
        # Get most similar jobs based on next text
        related_product_indices = cosine_similarities.argsort()[:-500:-1]
        all_books = self.reviews.iloc[related_product_indices]
        all_books = all_books[all_books["genres"].apply(lambda x: any(k in tags.values() for k in x.keys()))]
        my_books_indecies = all_books["average_rating"].argsort()
        my_books = all_books.iloc[my_books_indecies]
        
        return my_books



In [ ]:
my_model1 = transform()

In [45]:
retriveObj = model(my_model1.reviews, my_model1.tfidf_wm, my_model1.tfidf_tokens)

In [46]:
retriveObj.retrive("adventure science physical", {0:"mystery, thriller, crime"})

,book_id,review_id,review_text,n_votes,title,image_url,url,average_rating,description,genres
151455,7048800,e46fb422efafcac8caf664bb93550e57,****THIS REVIEW CONTAINS SPOILERS**** \n Every...,243,The Particular Sadness of Lemon Cake,https://images.gr-assets.com/books/1320541766m...,https://www.goodreads.com/book/show/7048800-th...,3.19,The wondrous Aimee Bender conjures the lush an...,"{'fiction': 1890, 'fantasy, paranormal': 370, ..."
76779,24751478,42da2d6170ae369a0a5d1cead2963ad8,So what is Soundless? \n A modern reinvention ...,31,Soundless,https://images.gr-assets.com/books/1446266199m...,https://www.goodreads.com/book/show/24751478-s...,3.36,In a village without sound...\nFor as long as ...,"{'fantasy, paranormal': 1071, 'young-adult': 6..."
618143,25489094,f371c3fe0d07718ceaf830d1868e277b,Three stories woven with common themes: Portug...,22,The High Mountains of Portugal,https://images.gr-assets.com/books/1443034505m...,https://www.goodreads.com/book/show/25489094-t...,3.37,"In Lisbon in 1904, a young man named Tomas dis...","{'fiction': 612, 'history, historical fiction,..."
591804,7199027,b331a8ebf33444bb8297f039f8ae0225,"It's very true that if you do something well, ...",46,"Warrior (The Blades of the Rose, #1)",https://images.gr-assets.com/books/1264553344m...,https://www.goodreads.com/book/show/7199027-wa...,3.39,"To most people, the realm of magic is the stuf...","{'romance': 583, 'fantasy, paranormal': 589, '..."
710039,6428887,054adcfe243e1bff113f606948574649,Pirates! Huzzah! What could possibly ruin a ro...,29,Pirate Latitudes,https://images.gr-assets.com/books/1332304096m...,https://www.goodreads.com/book/show/6428887-pi...,3.41,Jamaica in 1665 is a rough outpost of the Engl...,"{'fiction': 1477, 'history, historical fiction..."
...,...,...,...,...,...,...,...,...,...,...
197197,25328893,2dbe5bc65123470f59a0c8112af52fe0,"Loved it! \n Lighter, more humorous with more ...",45,"Arm of the Sphinx (The Books of Babel, #2)",https://images.gr-assets.com/books/1428607168m...,https://www.goodreads.com/book/show/25328893-a...,4.56,"Forced by necessity into a life of piracy, Sen...","{'fantasy, paranormal': 106, 'fiction': 11, 'm..."
842173,17347379,c03bee9788f4c0cc825c32ceda8bad20,After 10 years of procrastination I've finally...,37,Harry Potter and the Deathly Hallows (Harry Po...,https://images.gr-assets.com/books/1375303674m...,https://www.goodreads.com/book/show/17347379-h...,4.62,Harry Potter is preparing to leave the Dursley...,"{'fantasy, paranormal': 42532, 'young-adult': ..."
865320,28937572,ddcbce942c6e728cf78121d277dfc214,5 stars. \n This is a book that makes you want...,27,"Crooked Kingdom (Six of Crows, #2)",https://images.gr-assets.com/books/1466791456m...,https://www.goodreads.com/book/show/28937572-c...,4.62,Crooked Kingdom: the highly anticipated sequel...,"{'fantasy, paranormal': 5028, 'young-adult': 2..."
131854,22299763,245044f19425f42cbbdc0012bcbd8d48,"I AM DESTROYED. Help, my heart is on the floor...",24,"Crooked Kingdom (Six of Crows, #2)",https://images.gr-assets.com/books/1456172607m...,https://www.goodreads.com/book/show/22299763-c...,4.62,Welcome to the world of the Grisha.\nAfter pul...,"{'fantasy, paranormal': 5028, 'young-adult': 2..."


In [47]:
from joblib import dump
dump(retriveObj, './../books_proj/SavedModels/my_model.pkl')

['./../books_proj/SavedModels/my_model.pkl']